In [1]:
import sys
import os

# Add the upstream directory to sys.path
upstream_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if upstream_dir not in sys.path:
    sys.path.insert(0, upstream_dir)

# Now you can import the module
from opentrons_api import ot2_api
from microtissue_manipulator import core, utils
import numpy as np 
import cv2
import time
import json
import keyboard
# from pynput import keyboard
from configs import paths
import matplotlib.pyplot as plt

In [2]:
cap = core.Camera(0)

Using camera without buffer ...
Camera initialized ...


In [3]:
openapi = ot2_api.OpentronsAPI()

In [7]:
openapi.toggle_lights()

In [8]:
openapi.home_robot()

Request status:
<Response [200]>
{
  "message": "Homing robot."
}


In [9]:
openapi.create_run(verbose=False)

In [4]:
_ = openapi.get_run_info()

Total number of runs: 20
Current run ID: 456d3037-5936-4c70-bf59-64b500fd4d6a
Current run status: idle


In [5]:
openapi.load_pipette()

Request status:
<Response [201]>
{
  "data": {
    "id": "1693918d-c4b2-4c9b-a226-f4abc9de4ad0",
    "createdAt": "2024-10-03T13:05:18.872339+00:00",
    "commandType": "loadPipette",
    "key": "1693918d-c4b2-4c9b-a226-f4abc9de4ad0",
    "status": "succeeded",
    "params": {
      "pipetteName": "p300_single_gen2",
      "mount": "left"
    },
    "result": {
      "pipetteId": "48aa3ae3-f94a-42fe-b265-09b1a0fcff11"
    },
    "startedAt": "2024-10-03T13:05:18.874358+00:00",
    "completedAt": "2024-10-03T13:05:21.021985+00:00",
    "intent": "setup",
    "notes": []
  }
}


In [12]:
_ = openapi.get_position()

{'x': 383.154913809626, 'y': 342.3747635258571, 'z': 200.9}


In [13]:
#Define a tip rack. This is the default tip rack for the robot.
TIP_RACK = "opentrons_96_tiprack_300ul"
#Load the tip rack. Slot = 1 by default.
openapi.load_labware(TIP_RACK, 9)

Labware ID:
1c2a8189-6735-4da9-9159-07135cda9d42



In [14]:
openapi.pick_up_tip(openapi.labware_dct['9'], "A1")

In [36]:
openapi.drop_tip(openapi.labware_dct['1'], "A1")

In [25]:
openapi.move_relative(axis="z", distance=1)

In [7]:
manual_movement = utils.ManualRobotMovement(openapi)

In [38]:
keyboard.unhook_all()

## Move around with camera

In [6]:
def on_mouse_click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print("Clicked at pixel coordinate: ({}, {})".format(x, y))


# Create an instance of the ManualRobotMovement class
manual_movement = utils.ManualRobotMovement(openapi)

window = cap.get_window()
cv2.setMouseCallback(cap.window_name, on_mouse_click)
while True:
    frame = cap.get_frame(undist=True)
    x, y, z = openapi.get_position(verbose=False).values()
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow(cap.window_name, frame)

    if cv2.waitKey(1) == ord('q'):
        keyboard.unhook_all()
        break

cv2.destroyAllWindows()

## Record robot positions and write to calibration.json

In [21]:
def on_mouse_click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print("Clicked at pixel coordinate: ({}, {})".format(x, y))


# Create an instance of the ManualRobotMovement class
manual_movement = utils.ManualRobotMovement(openapi)

window = cap.get_window()
cv2.setMouseCallback(cap.window_name, on_mouse_click)
while True:
    frame = cap.get_frame(undist=True)
    cv2.imshow(cap.window_name, frame)

    if cv2.waitKey(1) == ord('q'):
        keyboard.unhook_all()
        break

cv2.destroyAllWindows()

utils.check_calibration_config()

with open(paths.CALIBRATION_PATH, 'r') as json_file:
    calibration_data = json.load(json_file)

calibration_data['robot_coords'] = manual_movement.positions

with open(paths.CALIBRATION_PATH, 'w') as json_file:
    json.dump(calibration_data, json_file, indent=4)

## Record pixel coordinates and write to calibration.json

In [18]:
x, y, z = openapi.get_position(verbose=False).values()
x,y,z


(51.37760524236513, 225.2428967184273, 152.09999999999997)

In [19]:
openapi.move_to_coordinates((51,225,100), min_z_height=1, verbose=False)


squaresX=7
squaresY=5 
squareLength=0.022
markerLength=0.011
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
params = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, params)
board = cv2.aruco.CharucoBoard((squaresX, squaresY), squareLength, markerLength, aruco_dict)


window = cap.get_window()
while True:
 # Capture frame-by-frame
    frame = cap.get_frame(undist=True)

    marker_corners, marker_ids, _ = detector.detectMarkers(frame)
    # print(marker_corners)

    top_left =  []
    if marker_ids is not None:
        for i in range(len(marker_ids)):
            corner = marker_corners[i][0][0]  # Upper left corner
            top_left.append(corner)
            cv2.circle(frame, tuple(corner.astype(int)), 5, (0, 255, 0), -1)
            # cv2.putText(frame, str(marker_ids[i][0]), (int(corner[0]), int(corner[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, f"({int(corner[0])}, {int(corner[1])})", (int(corner[0]), int(corner[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Our operations on the frame come here
    cv2.imshow(cap.window_name, frame)
    if cv2.waitKey(1) == ord('q'):
        break
 
# When everything done, release the capture
# cap.release_camera()
cv2.destroyAllWindows()

top_left_array = np.array(top_left)
top_left_array.tolist()
x,y,z = openapi.get_position(verbose=False).values()

with open(paths.CALIBRATION_PATH, 'r') as json_file:
    calibration_data = json.load(json_file)

calibration_data['camera_coords'] = top_left_array.tolist()
calibration_data['calib_origin'] = [x,y,z]

with open(paths.CALIBRATION_PATH, 'w') as json_file:
    json.dump(calibration_data, json_file, indent=4)


## Write transformation matrix

In [20]:
with open(paths.CALIBRATION_PATH, 'r') as json_file:
    calibration_data = json.load(json_file)

camera_coords = calibration_data['camera_coords']
robot_coords = calibration_data['robot_coords']
robot_coords = np.array(robot_coords)[:,:2].tolist()

camera_coords = utils.sort_coordinates(camera_coords)
robot_coords = utils.sort_coordinates(robot_coords, reverse_y=True)

robot_to_camera_coords = {tuple(robot_coord): tuple(camera_coord) for robot_coord, camera_coord in zip(robot_coords, camera_coords)}
tf_mtx = utils.compute_tf_mtx(robot_to_camera_coords)

calibration_data['tf_mtx'] = tf_mtx.tolist()

with open(paths.CALIBRATION_PATH, 'w') as json_file:
    json.dump(calibration_data, json_file, indent=4)

## Move and pick

In [23]:
with open(paths.CALIBRATION_PATH, 'r') as json_file:
    calibration_data = json.load(json_file)

tf_mtx = np.array(calibration_data['tf_mtx'])
calib_origin = np.array(calibration_data['calib_origin'])[:2]


def on_mouse_click(event, cX, cY, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print("Clicked at pixel coordinate: ({}, {})".format(cX, cY))
        X, Y, _ = tf_mtx @ (cX, cY, 1)

        x, y, _ = openapi.get_position(verbose=False).values()
        diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
        X += diff[0]
        Y += diff[1]
        print(f"Robot coords: ({x}, {y})")
        print(f"Clicked on: ({X}, {Y})")
        openapi.move_to_coordinates((X, Y, 10), min_z_height=1)

    if event == cv2.EVENT_RBUTTONDOWN:
        x, y, _ = openapi.get_position(verbose=False).values()
        openapi.move_to_coordinates((x, y, 100), min_z_height=1)


# Create an instance of the ManualRobotMovement class
manual_movement = utils.ManualRobotMovement(openapi)

window = cap.get_window()
cv2.setMouseCallback(cap.window_name, on_mouse_click)
while True:
    frame = cap.get_frame(undist=True)
    x, y, z = openapi.get_position(verbose=False).values()
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} cm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow(cap.window_name, frame)

    if cv2.waitKey(1) == ord('q'):
        keyboard.unhook_all()
        break

cv2.destroyAllWindows()

In [42]:
openapi.move_to_coordinates((X,Y,1), min_z_height=1)

Request status:
<Response [201]>
{
  "data": {
    "id": "1cf40a93-d882-422d-a5e9-fbbe3499c1f7",
    "createdAt": "2024-08-09T09:58:22.375613+00:00",
    "commandType": "moveToCoordinates",
    "key": "1cf40a93-d882-422d-a5e9-fbbe3499c1f7",
    "status": "succeeded",
    "params": {
      "minimumZHeight": 1.0,
      "forceDirect": false,
      "pipetteId": "77850757-0071-4ada-8396-42ec1ba1c667",
      "coordinates": {
        "x": 60.507194082642,
        "y": 306.7301624781195,
        "z": 1.0
      }
    },
    "result": {
      "position": {
        "x": 60.507194082642,
        "y": 306.7301624781195,
        "z": 1.0
      }
    },
    "startedAt": "2024-08-09T09:58:22.377527+00:00",
    "completedAt": "2024-08-09T09:58:24.084573+00:00",
    "intent": "setup",
    "notes": []
  }
}
